In [103]:
import os; os.environ['TF_CPP_MIN_LOG_LEVEL']='3'; 
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import metrics as skmetrics
import pickle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.utils.vis_utils import plot_model


**LOAD DATASETS**

In [60]:
filename='dataset_training.csv'
df_train=pd.read_csv(filename, nrows=4501, header=0,comment='#', na_values=[""])

In [61]:
df_train.head()

,dito1,dito2,dito3,dito4,target
0,3830,6240,4307,5058,0
1,3830,6240,4307,5058,0
2,3881,6208,4280,5041,0
3,3881,6208,4280,5041,0
4,3818,6089,4194,4965,0


In [76]:
# Data preparation
X_train = df_train.drop(['target'], axis = 1).values
y_train = df_train['target'].values

print("Input shape (X): {} --- Output shape (y): {}".format(X_train.shape, y_train.shape))

Input shape (X): (4500, 4) --- Output shape (y): (4500,)


In [77]:
print(X_train)

[[ 3830  6240  4307  5058]
 [ 3830  6240  4307  5058]
 [ 3881  6208  4280  5041]
 ...
 [ 6378 10103  7613  8403]
 [ 6407 10122  7623  8428]
 [ 6407 10122  7623  8428]]


In [78]:
print(y_train)

[0 0 0 ... 2 2 2]


In [79]:
filename='dataset_test.csv'
df_test=pd.read_csv(filename, nrows=1501, header=0,comment='#', na_values=[""])
df_test.head()

,dito1,dito2,dito3,dito4,target
0,4166,6040,4167,4657,0
1,4166,6040,4167,4657,0
2,4151,6033,4151,4632,0
3,4151,6033,4151,4632,0
4,4168,6035,4157,4648,0


In [80]:
# Data preparation
X_test = df_test.drop(['target'], axis = 1).values
y_test = df_test['target'].values

print("Input shape (X): {} --- Output shape (y): {}".format(X_test.shape, y_test.shape))

Input shape (X): (1500, 4) --- Output shape (y): (1500,)


In [81]:
print(X_test)
print(y_test)

[[ 4166  6040  4167  4657]
 [ 4166  6040  4167  4657]
 [ 4151  6033  4151  4632]
 ...
 [ 4733 10066  8074  8211]
 [ 4783 10123  8209  8271]
 [ 4783 10123  8209  8271]]
[0 0 0 ... 2 2 2]


**MODEL DEFINITION**

In [82]:
input = tf.keras.layers.Input(shape=(4500,4), batch_size= 150)

D1 = tf.keras.layers.Dense(units=64, activation='relu', use_bias=True)(input)

D2 = tf.keras.layers.Dense(units=32, activation='relu', use_bias=True)(D1)

output = tf.keras.layers.Dense(units=3, activation='softmax')(D2)

In [83]:
model= tf.keras.Model(inputs= input, outputs=output, name= "model")

In [84]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(150, 4500, 4)]          0         
                                                                 
 dense_6 (Dense)             (150, 4500, 64)           320       
                                                                 
 dense_7 (Dense)             (150, 4500, 32)           2080      
                                                                 
 dense_8 (Dense)             (150, 4500, 3)            99        
                                                                 
Total params: 2,499
Trainable params: 2,499
Non-trainable params: 0
_________________________________________________________________


**COMPILATION**

*Custom Loss*

In [86]:
bce = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.SUM)
def custom_crossentropy(bce):
    def loss(y_train, y_pred):
      
      #y_train = y_train[:,0:2]
      
      unweighted_loss = bce(y_train, y_pred)
      return unweighted_loss  

    return loss

*Custom Metrics*

In [88]:
def recall(y_train, y_pred):
    true_positives = K.sum(K.round(K.clip(y_train * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_train, 0, 1)))
    recall_v = true_positives / (possible_positives + K.epsilon())
    return recall_v

def precision(y_train, y_pred):
    true_positives = K.sum(K.round(K.clip(y_train * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision_v = true_positives / (predicted_positives + K.epsilon())
    return precision_v

def f1score(y_train, y_pred):
    precision_v = precision(y_train, y_pred)
    recall_v = recall(y_train, y_pred)
    return 2*((precision_v*recall_v)/(precision_v+recall_v+K.epsilon()))

*Model*

In [91]:
adam=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(
    loss= custom_crossentropy(bce),
    optimizer=adam,
    metrics=['acc',f1score,precision, recall]
)

In [99]:
plot_model(model,show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


**Checkpoints definition MODEL1**

In [100]:
earlystop_patience = 10 # 
earlystopper = EarlyStopping(patience=earlystop_patience, verbose=1)

In [102]:
#root = '\Users\Windows\Desktop\Magistrale\II_primo_semestre\Project  Lab Sens\AY2223_I_Project-3\PYTHON'
BASE_DIR = os.getcwd()
CALLBACKS_FOLDER =  os.path.join(BASE_DIR, 'MODEL_no_res')
os.makedirs(CALLBACKS_FOLDER, exist_ok=True)

In [105]:
# Example of checkpointer based on values of F1-score in the validation set 
checkpointer_val_f1score = ModelCheckpoint(filepath=CALLBACKS_FOLDER + "/best_max_val_f1score_no_res.h5",
                                          monitor='val_f1score',
                                          verbose=1,
                                          save_best_only=True,
                                          mode='max',
                                          period=1)


**MODEL TRAINIG**

In [106]:
steps = 4500//150
history1 = model.fit(X_train, #Give the generator as input data
          validation_data = (X_test,y_test), #Give directly all images and labels for the validation set. 
          epochs = 30, steps_per_epoch = steps,
          callbacks =[earlystopper,checkpointer_val_f1score], #Multiple checkpointer can be considered
          verbose =1)

Epoch 1/30


ValueError: in user code:

    File "c:\Users\Windows\Desktop\Magistrale\venv\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Windows\Desktop\Magistrale\venv\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Windows\Desktop\Magistrale\venv\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Windows\Desktop\Magistrale\venv\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Windows\Desktop\Magistrale\venv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Windows\Desktop\Magistrale\venv\lib\site-packages\keras\engine\input_spec.p